# Chapter 18: Monad

## Write `Monad` instances for the following types

In [3]:
:info Monad

class Applicative m => Monad (m :: * -> *) where
  (>>=) :: m a -> (a -> m b) -> m b
  (>>) :: m a -> m b -> m b
  return :: a -> m a
  fail :: String -> m a
  {-# MINIMAL (>>=) #-}
  	-- Defined in ‘GHC.Base’
instance Monad IHaskellTH.Q -- Defined in ‘Language.Haskell.TH.Syntax’
instance Monad (Either e) -- Defined in ‘Data.Either’
instance Monad [] -- Defined in ‘GHC.Base’
instance Monad Maybe -- Defined in ‘GHC.Base’
instance Monad IO -- Defined in ‘GHC.Base’
instance Monad ((->) r) -- Defined in ‘GHC.Base’
instance Monoid a => Monad ((,) a) -- Defined in ‘GHC.Base’

In [4]:
:info Applicative

class Functor f => Applicative (f :: * -> *) where
  pure :: a -> f a
  (<*>) :: f (a -> b) -> f a -> f b
  GHC.Base.liftA2 :: (a -> b -> c) -> f a -> f b -> f c
  (*>) :: f a -> f b -> f b
  (<*) :: f a -> f b -> f a
  {-# MINIMAL pure, ((<*>) | liftA2) #-}
  	-- Defined in ‘GHC.Base’
instance Applicative IHaskellTH.Q -- Defined in ‘Language.Haskell.TH.Syntax’
instance Applicative (Either e) -- Defined in ‘Data.Either’
instance Applicative [] -- Defined in ‘GHC.Base’
instance Applicative Maybe -- Defined in ‘GHC.Base’
instance Applicative IO -- Defined in ‘GHC.Base’
instance Applicative ((->) a) -- Defined in ‘GHC.Base’
instance Monoid a => Applicative ((,) a) -- Defined in ‘GHC.Base’

In [5]:
:info Functor

class Functor (f :: * -> *) where
  fmap :: (a -> b) -> f a -> f b
  (<$) :: a -> f b -> f a
  {-# MINIMAL fmap #-}
  	-- Defined in ‘GHC.Base’
instance Functor IHaskellTH.Q -- Defined in ‘Language.Haskell.TH.Syntax’
instance Functor (Either a) -- Defined in ‘Data.Either’
instance Functor [] -- Defined in ‘GHC.Base’
instance Functor Maybe -- Defined in ‘GHC.Base’
instance Functor IO -- Defined in ‘GHC.Base’
instance Functor ((->) r) -- Defined in ‘GHC.Base’
instance Functor ((,) a) -- Defined in ‘GHC.Base’

In [8]:
data Nope a = NopeDotJpg

instance Functor Nope where
    fmap f _ = NopeDotJpg

instance Applicative Nope where
    pure _ = NopeDotJpg

instance Monad Nope where
    NopeDotJpg >>= f = NopeDotJpg

In [13]:
data PhhhbbtttEither b a
    = Left a
    | Right b
    
instance Functor (PhhhbbtttEither b) where
    fmap a2a' (Left a) = Left (a2a' a)
    fmap _ (Right b) = Right b
    
instance Applicative (PhhhbbtttEither b) where
    pure = Left
    
instance Monad (PhhhbbtttEither b) where
    (Right x) >>= _ = Right x
    (Left x) >>= f = f x

In [14]:
newtype Identity a = Identity a deriving (Eq, Ord, Show)

instance Functor Identity where
    fmap f (Identity x) = Identity (f x)
    
instance Applicative Identity where
    pure = Identity
    
instance Monad Identity where
    (Identity x) >>= f = f x

In [20]:
data List a
    = Nil
    | Cons a (List a)
    
instance Functor List where
    fmap f Nil = Nil
    fmap f (Cons x xs) = Cons (f x) (fmap f xs)
    
instance Applicative List where
    pure x = Cons x Nil
    
instance Monad List where
    Nil >>= _ = Nil
    a@(Cons x xs) >>= f = accumulate . fmap f $ a where
        accumulate Nil = Nil
        accumulate (Cons Nil xs) = accumulate xs
        accumulate (Cons (Cons xx xxs) xs) = Cons xx (accumulate (Cons xxs xs))

## Write the following functions using the methods provided by Monad and Functor

In [23]:
j :: Monad m => m (m a) -> m a
j = (>>= id)

In [25]:
j [[1, 2], [], [3]]

[1,2,3]

In [26]:
j (Just (Just 1))

Just 1

In [27]:
j (Just Nothing)

Nothing

In [29]:
j Nothing

Nothing

In [31]:
l1 :: Monad m => (a -> b) -> m a -> m b
l1 = fmap

In [52]:
l2 :: Monad m => (a -> b -> c) -> m a -> m b -> m c
l2 f x y = goal where
    f' = fmap (fmap . f)
    goal = f' x >>= ($ y)

In [55]:
a :: Monad m => m a -> m (a -> b) -> m b
a x f = f >>= (<$> x)

In [58]:
meh :: Monad m => [a] -> (a -> m b) -> m [b]
meh [] _ = pure []
meh (x:xs) f = goal where
    x' = f x
    glue = l2 (:)
    goal = glue x' (meh xs f)    

In [59]:
flipType :: Monad m => [m a] -> m [a]
flipType xs = meh xs id